# Healthy Trees, Healthy Lives Analysis
## Notes


## Variables and Boundary Creation - Do this if you need to re-run a section of the analysis or if you restart the kernal

### Initialize Varibles

In [ ]:
import os
arcpy.env.overwriteOutput = True
 
# Set workspace and input variables. CHANGE THESE DEPENDING ON CITY
City_Name = "MissouriCity"
City_Name_With_Spaces = "Missouri City"
workspace = r"D:\ArcGIS_Projects\HTHL_Gorman\HTHL"
#Add these depending on the city
parcels = "Miss_parcels"
utc = "MissouriCity_utc"

#Add these from the Base_Polygons GDB. These DO NOT change
city_points = "Cities_Points"
city_boundaries = "Cities"
point_data = "School_MedicalFacilities_Texas" 
building_polygons = "Texas_Building_Polygons" 
streets = "TxDOT_Roadway_Inventory"
parks = "PAD_US"

# Create a new geodatabase based on the city name
gdb_name = f"{City_Name}.gdb"
gdb_path = os.path.join(workspace, gdb_name)
arcpy.env.workspace = gdb_path
spatial_ref = arcpy.SpatialReference(3665)
print(f"{City_Name} Initialized")

### Create GDB if needed and make a city boundary layer

In [ ]:
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(workspace, gdb_name)
# Extract the city boundary
city_boundary_layer = "CityBoundaryLayer"
city_point_layer = "CityPointLayer"
arcpy.MakeFeatureLayer_management(city_boundaries, city_boundary_layer, f'"CITY_NM" = \'{City_Name}\'')
arcpy.MakeFeatureLayer_management(city_points, city_point_layer, f'"CITY_NM" = \'{City_Name}\'')
city_boundary_output = os.path.join(gdb_path, f"{City_Name}_Boundary")
city_point_output = os.path.join(gdb_path, f"{City_Name}_Point")
arcpy.CopyFeatures_management(city_boundary_layer, city_boundary_output)
arcpy.CopyFeatures_management(city_point_layer, city_point_output)
print(f"{City_Name} GDB Created")

## Part 1 - Schools and Medical Facilities

### a) Clip points and spatially join to building polygons

In [ ]:
# Clip the point data to the city boundary
clipped_points = os.path.join(gdb_path, f"{City_Name}_Clipped_Points")
arcpy.Clip_analysis(point_data, city_boundary_output, clipped_points)
print("Points Created for City")

In [ ]:
# clipped_points = os.path.join(gdb_path, f"{City_Name}_Clipped_Points")
# city_boundary_output = os.path.join(gdb_path, f"{City_Name}_Boundary")
# city_point_output = os.path.join(gdb_path, f"{City_Name}_Point")

In [ ]:
# Count the number of clipped points
point_count = arcpy.GetCount_management(clipped_points)
print(f"Number of clipped points: {point_count[0]}")

# Spatial join between the clipped points and building polygons
spatial_join_output = os.path.join(gdb_path, f"{City_Name}_Spatial_Join")
arcpy.SpatialJoin_analysis(building_polygons, clipped_points, spatial_join_output, join_type="KEEP_COMMON")

# Count the number of polygons after the spatial join
arcpy.DeleteIdentical_management(f"{City_Name}_Spatial_Join", ["Name"])
polygon_count = arcpy.GetCount_management(spatial_join_output)
# Use the appropriate field(s) for duplicates
print(f"Number of polygons after spatial join: {polygon_count[0]}")

# Validate counts
if point_count[0] == polygon_count[0]:
    print("The point count matches the polygon count.")
else:
    print("The point count does NOT match the polygon count. Fix and run again until the polygon count matches")

### c) Bulk School/Medical Analysis. Please review the README file for more information on logic. 

In [ ]:
building_projected = arcpy.management.Project(spatial_join_output, f"{City_Name}_Buildings_Projected", spatial_ref)
arcpy.management.AddFields(
    building_projected,
    [["HTHL_Type", "TEXT", "HTHL_Type"]])
arcpy.management.CalculateField(building_projected, "HTHL_Type", "'School/Medical'", expression_type = "PYTHON3")

building_buffer = arcpy.analysis.Buffer(building_projected, f"{City_Name}_Building_Buffer", "200 Meters", dissolve_option = "NONE")
building_erase = arcpy.analysis.Erase(building_buffer, building_projected, f"{City_Name}_Building_Erase")

arcpy.management.AddFields(
    building_erase,
    [
        ["buffer_acres", 'FLOAT', 'buffer_acres']])
arcpy.management.CalculateGeometryAttributes(building_erase, geometry_property = [["buffer_acres", "AREA_GEODESIC"]], area_unit = "ACRES_US", coordinate_system = spatial_ref)

building_utc_unprojected = arcpy.analysis.Clip(building_buffer, utc , f"{City_Name}_UTC_Unprojected")
building_utc = arcpy.management.Project(building_utc_unprojected, f"{City_Name}_UTC_Projected", spatial_ref)

arcpy.management.AddFields(
    building_utc,
    [
        ["utc_acres", 'FLOAT', 'utc_acres'],
        ["HTHL_Type", "TEXT", "HTHL_Type"],
        ["TreeCanopyPercentage", "FLOAT", "TreeCanopyPercentage"]
    ])
arcpy.management.CalculateGeometryAttributes(building_utc, geometry_property = [["utc_acres", "AREA_GEODESIC"]], area_unit = "ACRES_US", coordinate_system = spatial_ref)
arcpy.management.JoinField(building_utc, "Name", building_erase, "Name", ["buffer_acres"])

arcpy.management.CalculateField(building_utc, "HTHL_Type", "'Tree Canopy'")
arcpy.management.CalculateField(building_utc, "TreeCanopyPercentage", '(!utc_acres! / !buffer_acres!) * 100', expression_type = "PYTHON3")

arcpy.management.JoinField(building_projected, "Name", building_utc, "Name", ["TreeCanopyPercentage"])
final_buildings = arcpy.management.Merge([building_utc, building_projected], f"{City_Name}_School_Medical_Final")
print(f"{City_Name} School/Medical Created")

## d) Table Reorganiztion

In [ ]:
fields_to_keep = ['Name', 'FullAddress', 'Category', 'TreeCanopyPercentage', 'Subcategory', 'HTHL_Type', 'Shape_Area', 'Shape_Length', "OBJECTID", 'Shape']

# Get all fields in the feature class
all_fields = [field.name for field in arcpy.ListFields(final_buildings)]

# Identify the fields to delete (those not in the 'fields_to_keep' list)
fields_to_delete = [field for field in all_fields if field not in fields_to_keep]

# Delete the fields that are not in the 'fields_to_keep' list
if fields_to_delete:
    arcpy.management.DeleteField(final_buildings, fields_to_delete)
    print(f"Deleted fields: {fields_to_delete}")
else:
    print("No fields to delete.")
arcpy.management.AddField(final_buildings, 'City', 'TEXT')
arcpy.management.CalculateField(final_buildings, 'City', f'"{City_Name_With_Spaces}"')
print(f"{City_Name} School/Medical Completed")

## Part 2 - Rights of Way (ROW)

### a) This block performs the bulk of the ROW analysis. Please review the README file for more information on the logic

In [ ]:
streets_unprojected = arcpy.analysis.Clip(streets, city_boundary_output, f"{City_Name}_Streets_Clip")
streets_projected = arcpy.management.Project(streets_unprojected, f"{City_Name}_Streets_Projected", spatial_ref)
streets_dissolved = arcpy.management.Dissolve(streets_projected, f"{City_Name}_Streets_Dissolved" , dissolve_field = "Street")
streets_buffered = arcpy.analysis.Buffer(streets_dissolved, f"{City_Name}_Streets_Buffer", "55 Feet", dissolve_option = "NONE")
streets_intersected = arcpy.analysis.Intersect([[utc, 2], [streets_buffered, 1]], f"{City_Name}_Streets_Intersect")

streets_intersect_dissolve = arcpy.management.Dissolve(streets_intersected, f"{City_Name}_Streets_Intersect_Dissolved" , dissolve_field = "Street")

arcpy.management.AddFields(
    streets_intersect_dissolve,  
    [["utc_acres", "FLOAT", "utc_acres"],
    ["HTHL_Type", "TEXT", "HTHL_Type"]])

arcpy.management.AddFields(
    streets_buffered,  
    [["HTHL_Type", "TEXT", "HTHL_Type"],
    ["TreeCanopyPercentage", "FLOAT", "TreeCanopyPercentage"],
     ["buffer_acres", "FLOAT", "buffer_acres"]])

arcpy.management.CalculateField(streets_buffered, "HTHL_Type", "'Street'", "PYTHON3")
arcpy.management.CalculateField(streets_intersect_dissolve, "HTHL_Type", "'Tree Canopy'", "PYTHON3")

arcpy.management.CalculateGeometryAttributes(streets_intersect_dissolve, geometry_property = [["utc_acres", "AREA_GEODESIC"]], area_unit = "ACRES_US", coordinate_system = spatial_ref)
arcpy.management.CalculateGeometryAttributes(streets_buffered, geometry_property = [["buffer_acres", "AREA_GEODESIC"]], area_unit = "ACRES_US", coordinate_system = spatial_ref)
arcpy.management.JoinField(streets_buffered, "Street", streets_intersect_dissolve, "Street", "utc_acres")
arcpy.management.SelectLayerByAttribute(f"{City_Name}_Streets_Buffer", "NEW_SELECTION", "utc_acres IS NULL")

arcpy.management.CalculateField(f"{City_Name}_Streets_Buffer", "utc_acres", 0 , "PYTHON3")
arcpy.management.SelectLayerByAttribute(f"{City_Name}_Streets_Buffer", "CLEAR_SELECTION")
arcpy.management.CalculateField(
    f"{City_Name}_Streets_Buffer",
    "TreeCanopyPercentage",
    '0 if !utc_acres! == 0 or !utc_acres! == None else (!utc_acres! / !buffer_acres!) * 100',
    "PYTHON3"
)

final_ROW = arcpy.management.Merge([streets_buffered, streets_intersect_dissolve], f"{City_Name}_ROW_Final")
arcpy.management.AddFields(
    final_ROW,  
    [["City", "TEXT", "City"]])
arcpy.management.CalculateField(f"{City_Name}_ROW_Final", "City", f'"{City_Name_With_Spaces}"', "PYTHON3")
print(f"{City_Name} ROW Created")

### b) Table Reorganization

In [ ]:
fields_to_keep = ['STREET', 'TreeCanopyPercentage', 'City', 'HTHL_Type', 'Shape_Area', 'Shape_Length', "OBJECTID", 'Shape']

# Get all fields in the feature class
all_fields = [field.name for field in arcpy.ListFields(final_ROW)]

# Identify the fields to delete (those not in the 'fields_to_keep' list)
fields_to_delete = [field for field in all_fields if field not in fields_to_keep]

# Delete the fields that are not in the 'fields_to_keep' list
if fields_to_delete:
    arcpy.management.DeleteField(final_ROW, fields_to_delete)
    print(f"Deleted fields: {fields_to_delete}")
else:
    print("No fields to delete.")
    
print(f"{City_Name} ROW Completed")

## Part 3 - Residential Areas

In [ ]:
parcel_clip = arcpy.analysis.Clip(parcels, city_boundary_output, f"{City_Name}_Parcels")
parcels_projected = arcpy.management.Project(parcel_clip, f"{City_Name}_Parcels_Projected", spatial_ref)
parks_projected = arcpy.management.Project(parks, f"{City_Name}_Parks_Projected", spatial_ref)
parks_buffer = arcpy.analysis.Buffer(parks_projected, f"{City_Name}_Parks_Buffer", "500 Meters", dissolve_option = "NONE")
parcels_within_500m = arcpy.analysis.Clip(parcels_projected, parks_buffer, f"{City_Name}_Parcels_Within_500m_FromParks")
parcels_over_500m = arcpy.analysis.Erase(parcels_projected, parcels_within_500m,  f"{City_Name}_Parcels_Over_500m_FromParks")
print(f"{City_Name} Parcels Within/Over 500m of Parks Created")

In [ ]:
# Initial fields to keep without including OBJECTID fields initially
fields_to_keep = ['OWNER_NAME', 'LEGAL_DESC', 'Category', 'SITUS_ADDR', 'Shape_Area', 'Shape_Length', 'Shape']

# Get all fields in the feature class
all_fields = [field.name for field in arcpy.ListFields(parcels_over_500m)]

# Try to delete both OBJECTID fields to see which one is non-deletable
test_objectid_fields = ['OBJECTID', 'OBJECTID_1', 'OBJECTID_12']
undeletable_field = None

for field in test_objectid_fields:
    try:
        arcpy.management.DeleteField(parcels_over_500m, field)
    except arcpy.ExecuteError:
        # This field is non-deletable, so we keep it
        undeletable_field = field
        break

# Add the non-deletable OBJECTID field to fields_to_keep
if undeletable_field:
    fields_to_keep.append(undeletable_field)
    print(f"Non-deletable field identified: {undeletable_field}")

# Identify fields to delete (those not in fields_to_keep list)
fields_to_delete = [field for field in all_fields if field not in fields_to_keep]

# Delete the fields that are not in the fields_to_keep list
if fields_to_delete:
    arcpy.management.DeleteField(parcels_over_500m, fields_to_delete)
    print(f"Deleted fields: {fields_to_delete}")
else:
    print("No fields to delete.")

# Rename OWNER_NAME to Name and SITUS_ADDR to Address
arcpy.management.AlterField(parcels_over_500m, 'OWNER_NAME', 'Owner')
arcpy.management.AlterField(parcels_over_500m, 'SITUS_ADDR', 'Address')
print("Fields renamed: OWNER_NAME to Name, SITUS_ADDR to Address")

# Add and calculate the 'City' field
arcpy.management.AddField(parcels_over_500m, 'City', 'TEXT')
city_value = f'"{City_Name_With_Spaces}"'  # Ensure City_Name_With_Spaces is a properly formatted string
arcpy.management.CalculateField(parcels_over_500m, 'City', city_value, "PYTHON3")
print(f"{City_Name} Parcels Over 500m of Parks Complete")

In [ ]:
# Initial fields to keep without including OBJECTID fields initially
fields_to_keep = ['OWNER_NAME', 'LEGAL_DESC', 'Category', 'SITUS_ADDR', 'Shape_Area', 'Shape_Length', 'Shape']

# Get all fields in the feature class
all_fields = [field.name for field in arcpy.ListFields(parcels_within_500m)]

# Try to delete both OBJECTID fields to see which one is non-deletable
test_objectid_fields = ['OBJECTID', 'OBJECTID_1', 'OBJECTID_12']
undeletable_field = None

for field in test_objectid_fields:
    try:
        arcpy.management.DeleteField(parcels_within_500m, field)
    except arcpy.ExecuteError:
        # This field is non-deletable, so we keep it
        undeletable_field = field
        break

# Add the non-deletable OBJECTID field to fields_to_keep
if undeletable_field:
    fields_to_keep.append(undeletable_field)
    print(f"Non-deletable field identified: {undeletable_field}")

# Identify fields to delete (those not in fields_to_keep list)
fields_to_delete = [field for field in all_fields if field not in fields_to_keep]

# Delete the fields that are not in the fields_to_keep list
if fields_to_delete:
    arcpy.management.DeleteField(parcels_within_500m, fields_to_delete)
    print(f"Deleted fields: {fields_to_delete}")
else:
    print("No fields to delete.")
    
# Rename OWNER_NAME to Name and SITUS_ADDR to Address
arcpy.management.AlterField(parcels_within_500m, 'OWNER_NAME', 'Owner')
arcpy.management.AlterField(parcels_within_500m, 'SITUS_ADDR', 'Address')
print("Fields renamed: OWNER_NAME to Name, SITUS_ADDR to Address")

# Add and calculate the 'City' field
arcpy.management.AddField(parcels_within_500m, 'City', 'TEXT')
arcpy.management.CalculateField(parcels_within_500m, 'City', f'"{City_Name_With_Spaces}"')
print(f"{City_Name} Parcels Within 500m of Parks Complete")

## Dissolve Parcels

In [ ]:
# This cell dissolves the parcels into single layers, one for parcels outside the 500m radius, and within the 500m radius. 
# This is so users can still see the overview of parcels for the city without having to load every parcel for the city

arcpy.management.Dissolve(parcels_within_500m, f"{City_Name}_Parcels_Within_500m_Dissolve", "City")
arcpy.management.Dissolve(parcels_over_500m, f"{City_Name}_Parcels_Over_500m_Dissolve", "City")

In [ ]:
def calculate_city_name(input_layer, field_name="CITY_NM"):
    """
    Populates the CITY_NM field in the given layer.
    """
    try:
        # Check if the field exists
        fields = [f.name for f in arcpy.ListFields(input_layer)]
        if field_name not in fields:
            raise ValueError(f"Field {field_name} does not exist in {input_layer}")
        
        # Start editing session if needed (for file geodatabases or enterprise GDBs)
        with arcpy.da.UpdateCursor(input_layer, [field_name]) as cursor:
            for row in cursor:
                # Example logic: Replace this with actual logic to determine CITY_NM
                row[0] = City_Name_With_Spaces   # Replace with actual city name determination logic
                cursor.updateRow(row)
        
        print(f"Successfully updated {field_name} in {input_layer}")
    except Exception as e:
        print(f"Error: {e}")

# Example usage
calculate_city_name(city_boundary_output)
calculate_city_name(city_point_output)


## Part 4 - Append Existing AGOL HTHL Layer

In [ ]:
building_appendlayer = "https://services5.arcgis.com/ELI1iJkCzTIagHkp/arcgis/rest/services/School_Medical_ComboTest/FeatureServer/0"
row_appendlayer = "https://services5.arcgis.com/ELI1iJkCzTIagHkp/arcgis/rest/services/Rgihts_of_Way/FeatureServer/0"
residential_within500_appendlayer = "https://services5.arcgis.com/ELI1iJkCzTIagHkp/arcgis/rest/services/Parcels_Within_500m_From_ParksandGreenspaces/FeatureServer/0"
residential_over500_appendlayer = "https://services5.arcgis.com/ELI1iJkCzTIagHkp/arcgis/rest/services/Parcels_500m_From_ParksandGreenspaces/FeatureServer/0"
residential_within500_dissolve_appendlayer = "https://services5.arcgis.com/ELI1iJkCzTIagHkp/arcgis/rest/services/Parcels_Within500m_FromParksandGreenspaces_Dissolved/FeatureServer/0"
residential_over500_dissolve_appendlayer = "https://services5.arcgis.com/ELI1iJkCzTIagHkp/arcgis/rest/services/Parcels_Over500m_FromParksandGreenspaces_Dissolved/FeatureServer/0"
city_points_append_layer = "https://services5.arcgis.com/ELI1iJkCzTIagHkp/arcgis/rest/services/HTHL_AvailableCityPoints/FeatureServer/0"
city_boundary_append_layer ="https://services5.arcgis.com/ELI1iJkCzTIagHkp/arcgis/rest/services/HTHL_CityBoundaries/FeatureServer/0"

arcpy.management.Append([final_buildings], building_appendlayer, schema_type = "NO_TEST", update_geometry = "UPDATE_GEOMETRY")
arcpy.management.Append([final_ROW], row_appendlayer, schema_type = "NO_TEST", update_geometry = "UPDATE_GEOMETRY")
arcpy.management.Append([parcels_within_500m], residential_within500_appendlayer, schema_type = "NO_TEST", update_geometry = "UPDATE_GEOMETRY")
arcpy.management.Append([parcels_over_500m], residential_over500_appendlayer, schema_type = "NO_TEST", update_geometry = "UPDATE_GEOMETRY")
arcpy.management.Append([f"{City_Name}_Parcels_Within_500m_Dissolve"], residential_within500_dissolve_appendlayer, schema_type = "NO_TEST", update_geometry = "UPDATE_GEOMETRY")
arcpy.management.Append([f"{City_Name}_Parcels_Over_500m_Dissolve"], residential_over500_dissolve_appendlayer, schema_type = "NO_TEST", update_geometry = "UPDATE_GEOMETRY")
arcpy.management.Append(city_point_output, city_points_append_layer, schema_type = "NO_TEST", update_geometry = "UPDATE_GEOMETRY")
arcpy.management.Append(city_boundary_output, city_boundary_append_layer, schema_type = "NO_TEST", update_geometry = "UPDATE_GEOMETRY")

print(f"{City_Name} Appended")

## Delete temporary layers, but keep the edited points and spatial joins just in case you need to go back and add more. Be sure the city was appended correctly with no issues before you delete. 

## You may have to run this twice. Sometimes the layers don't actually delete, especially if it says you've saved 0 MB.

In [ ]:
import os
# Set the project and map
aprx = arcpy.mp.ArcGISProject("CURRENT")  # Uses the currently open ArcGIS Pro project
m = aprx.activeMap  # Gets the active map

# Remove all layers
for layer in m.listLayers():
    m.removeLayer(layer)
    
keep_list = [
    f"{City_Name}_Clipped_Points",
    f"{City_Name}_Parcels_Over_500m_FromParks",
    f"{City_Name}_Parcels_Within_500m_FromParks",
    f"{City_Name}_ROW_Final",
    f"{City_Name}_School_Medical_Final",
    f"{City_Name}_Spatial_Join"
]
# Function to calculate the size of a folder (geodatabase)
def get_gdb_size(gdb_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(gdb_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size
size_before = get_gdb_size(gdb_path)
size_before_mb = size_before / (1024 * 1024)
print(f"Geodatabase size: {size_before_mb:.2f} MB")

# Get a list of all feature classes in the geodatabase
feature_classes = arcpy.ListFeatureClasses()
print(feature_classes)
# # # Loop through feature classes and delete those not in the keep list
for fc in feature_classes:
    if fc not in keep_list:
        arcpy.Delete_management(fc)
        print(f"Deleted {fc}")
    else:
        print(f"Kept {fc}")

# Get the size in bytes, then convert to megabytes (MB)
size_after = get_gdb_size(gdb_path)
size_after_mb = size_after / (1024 * 1024)
print(f"GDB size After Deletion: {size_after_mb:.2f} MB\n")
size_savings = size_before_mb - size_after_mb
print(f"You saved {size_savings} MB of storage")